## Q & A on private documents

## Required setup

In [19]:
! pip install python-dotenv

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [35]:
! pip install openai

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..



  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/76.5 kB ? eta -:--:--
   ---------------------------------------- 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [20]:
! pip install tiktoken

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [21]:
! pip install pinecone-client

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [22]:
import os
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)
pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'),environment=os.environ.get('PINECONE_ENV'))
print("a",os.environ.get('PINECONE_API_KEY'))

a 932a7998-263d-49be-86c6-54eba5a99ec0


In [23]:
! pip install pypdf -q

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [24]:
! pip install langchain

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [25]:
def load_document(file):

    name, extension = os.path.splitext(file)

    if extension == '.pdf' :
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    else:
        print(f"Document format {extension} is not supported!")
        
    # print(data[1].page_content)
    data = loader.load()
    return data

def load_from_wikipedia(query, lang='en', load_max_docs=2):
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data
    


In [26]:
def chunk_data(data,chunk_size=256):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=0)
    chunks=text_splitter.split_documents(data)
    return chunks
    

In [27]:
def print_embeddings_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts] )
    print(f'Total Token: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens/1000 * 0.0004:.6f}')
    

In [37]:
def insert_or_fetch_embeddings(index_name):

    embeddings=OpenAIEmbeddings()
    

    if index_name in pinecone.list_indexes():
        print(f'index {index_name} already exists. Loading embeddings ...',end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('ok')
    else:
        print(f'Creating index {index_name} and embeddings ...',end='')
        pinecone.create_index(index_name, dimension=1536,metric='cosine')
        vector_store=Pinecone.from_documents(chunks,embeddings,index_name=index_name)
        print('ok')

    return vector_store

In [29]:
def delete_pinecone_index(index_name='all'):

    if index_name=='all':
        indexes = pinecone.list_indexes()
        print("Deleting allll indexes....")
        for index in indexes:
            pinecone.delete_index(index)
        print("ok")

    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('ok')
    

## Running the Modules

In [30]:
## Running code
data = load_document('./data/SwinUnet.pdf')
# data = load_document('./Llama2_QuamtizedRun.ipynb')
# print(data[1].page_content)
# print(data[10].metadata)

print(f'There are {len(data)} pages in the pdf document')


Loading ./data/SwinUnet.pdf
There are 14 pages in the pdf document


In [31]:
chunks = chunk_data(data)
print(len(chunks))
print("$$$$$")
print(chunks[2].page_content)


155
$$$$$
neural networks based on U-shaped architecture and skip-connections
have been widely applied in a variety of medical image tasks. How-
ever, although CNN has achieved excellent performance, it cannot learn


In [32]:
print_embeddings_cost(chunks)

Total Token: 9383
Embedding Cost in USD: 0.003753


In [33]:
delete_pinecone_index()

Deleting allll indexes....
ok


In [38]:
index_name = 'askswinunet'
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askswinunet and embeddings ...

TypeError: VectorStore.from_documents() takes 3 positional arguments but 4 were given

In [ ]:
! pip install wikipedia

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [ ]:
data = load_from_wikipedia('GPT-4')
print(data[0].page_content)

Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its series of GPT foundation models. It was initially released on March 14, 2023, and has been made publicly available via the paid chatbot product ChatGPT Plus, and via OpenAI's API.  As a transformer-based model, GPT-4 uses a paradigm where pre-training using both public data and "data licensed from third-party providers" is used to predict the next token. After this step, the model was then fine-tuned with reinforcement learning feedback from humans and AI for human alignment and policy compliance.: 2 Observers reported that the iteration of ChatGPT using GPT-4 was an improvement on the previous iteration based on GPT-3.5, with the caveat that GPT-4 retains some of the problems with earlier revisions. GPT-4 is also capable of taking images as input, though this feature has not been made available since launch. OpenAI has declined to reveal various technical details 